# Sensitivity analysis for two-component mixture-of-exponentials model

## Full dataset

Begin by running the model on the full dataset:

In [1]:
library(rstan)

load("data_m.RData")

Loading required package: ggplot2
Loading required package: StanHeaders
Warning message:
"package 'StanHeaders' was built under R version 3.2.5"rstan (Version 2.12.1, packaged: 2016-09-11 13:07:50 UTC, GitRev: 85f7a56811da)
For execution on a local, multicore CPU with excess RAM we recommend calling
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())


In [2]:
####################
# initial values
####################

# Initial values - 0
init0 <- list(psi = 7/77, lambda_slow = 0.74, p1 = 0.23)

# Initial values - 1
init1 <- list(psi = 0.9, lambda_slow = 0.7, p1 = 0.2)

# Initial values - 2
init2 <- list(psi = 0.6, lambda_slow = 0.1, p1 = 0.5)

####################
# run the model
####################

fit_m <- stan(file = 'chlamydia_two_exponentials_men.stan', 
              data = chlamydia_dat_m, 
              iter = 22000, warmup = 2000, init=list(init0, init1, init2), chains = 3, seed=12345,
              sample_file = 'results/sample_men.csv',
              verbose = FALSE
             )

op_m <- extract(fit_m)

In file included from filedb77ddbe889.cpp:8:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/src/stan/model/model_header.hpp:4:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/stan/math.hpp:4:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/stan/math/rev/mat.hpp:4:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/stan/math/rev/core.hpp:42:
/Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/stan/math/rev/core/set_zero_all_adjoints.hpp:14:17: warning: unused function 'set_zero_all_adjoints' [-Wunused-function]
    static void set_zero_all_adjoints() {
                ^
In file included from filedb77ddbe889.cpp:8:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/src/stan/model/mo

Warning message:
"There were 1 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup"Warning message:
"Examine the pairs() plot to diagnose sampling problems
"

In [ ]:
quantile(op_m$lambda_slow, p=c(0.5, 0.025, 0.975))

## Sensitivity to test method

Now repeat the analysis on only the studies using culture diagnosis methods, or only the studies using NAAT diagnosis methods.

### Culture diagnosis only

In [ ]:
load("data_m_culture.RData")

fit_m_culture <- stan(file = 'chlamydia_two_exponentials_men.stan', 
              data = chlamydia_dat_m_culture, 
              iter = 22000, warmup = 2000, init=list(init0, init1, init2), chains = 3, seed=12345,
              sample_file = 'results/sample_men_culture.csv',
              verbose = FALSE
             )

op_m_culture <- extract(fit_m_culture)

In [ ]:
quantile(op_m_culture$lambda_slow, p=c(0.5, 0.025, 0.975))

### NAAT diagnosis only

In [ ]:
load("data_m_naat.RData")

fit_m_naat <- stan(file = 'chlamydia_two_exponentials_men.stan', 
              data = chlamydia_dat_m_naat, 
              iter = 22000, warmup = 2000, init=list(init0, init1, init2), chains = 3, seed=12345,
              sample_file = 'results/sample_men_naat.csv',
              verbose = FALSE
             )

op_m_naat <- extract(fit_m_naat)

In [ ]:
quantile(op_m_naat$lambda_slow, p=c(0.5, 0.025, 0.975))

In [ ]:
options(repr.plot.width=7, repr.plot.height=4)
par(mar = c(5.1, 0.1, 0.1, 0.1))

plot_data <- rbind(
    quantile(op_m$lambda_slow, p=c(0.025, 0.25, 0.5, 0.75, 0.975)),
    quantile(op_m_culture$lambda_slow, p=c(0.025, 0.25, 0.5, 0.75, 0.975)),
    quantile(op_m_naat$lambda_slow, p=c(0.025, 0.25, 0.5, 0.75, 0.975))
    )

plot(plot_data[,'50%'], c(1,2,3), pch=16, xlim=c(-1.5,9), ylim=c(0,4),
     xlab=expression(Slow~clearance~rate~(years^{-1})), ylab='', yaxt='n')
arrows(x0=plot_data[,'25%'], x1=plot_data[,'75%'], y0=c(1,2,3), length=0, lwd=3)
arrows(x0=plot_data[,'2.5%'], x1=plot_data[,'97.5%'], y0=c(1,2,3), length=0)

text(0, 1, 'All studies', pos=2, cex=0.8)
text(0, 2, 'Culture studies', pos=2, cex=0.8)
text(0, 3, 'NAAT studies', pos=2, cex=0.8)

legend('topright', legend=c('Median', NA, '50% CrI', NA, '95% CrI'),
       lty=c(0,NA,1,NA,1), lwd=c(0,NA,3,NA,1), pch=c(16,NA,NA,NA,NA), 
       inset = 0.05, cex=0.8)

## Study quality and influential studies

In [ ]:
study_names <- c("Handsfield", "Prentice", "Johannisson", "Paavonen", "Joyner", "Geisler", "Stamm", "van den Brule")
op_m <- list()
cumobsplus <- c(chlamydia_dat_m$cumobs, chlamydia_dat_m$Nobs) # vector for use in making each new dataset in loop

In [ ]:
for(i in 1:chlamydia_dat_m$studnum){

    print(i)

    chlamydia_dat_tmp <- list(
        studnum = 7,
        studnum_bytype = chlamydia_dat_m$studnum_bytype - c(i <= 6, i > 6, 0),
        studobs = chlamydia_dat_m$studobs[-i],
        cumobs = c(0, cumsum(head(chlamydia_dat_m$studobs[-i], n=-1))),
        Nobs = sum(chlamydia_dat_m$studobs[-i]),
        r = chlamydia_dat_m$r[-((cumobsplus[i]+1):(cumobsplus[i+1]))],
        n = chlamydia_dat_m$n[-((cumobsplus[i]+1):(cumobsplus[i+1]))],
        t = chlamydia_dat_m$t[-((cumobsplus[i]+1):(cumobsplus[i+1]))],
        seind = chlamydia_dat_m$seind[-((cumobsplus[i]+1):(cumobsplus[i+1]))],
        T = rep(999, times = sum(chlamydia_dat_m$studobs[-i]))
        )
    
    ####################
    # run the model
    ####################
    
    fit_m <- stan(file = 'chlamydia_two_exponentials_men.stan', 
                data = chlamydia_dat_tmp, 
                iter = 22000, warmup = 2000, init=list(init0, init1, init2), chains = 3, seed=12345
                 )

    op_m[[study_names[i]]] <- extract(fit_m)

    }

In [ ]:
pmeans <- NULL
pquantiles <- NULL
for(i in 1:chlamydia_dat_m$studnum){

#    print(study_names[[i]])
#    print(mean(op_m[[i]]$lambda_slow))
    pmeans <- c(pmeans, mean(op_m[[i]]$lambda_slow))
#    print(quantile(op_m[[i]]$lambda_slow, p=c(0.5, 0.25, 0.75, 0.025, 0.975)))
    pquantiles <- rbind(pquantiles, quantile(op_m[[i]]$lambda_slow, p=c(0.5, 0.25, 0.75, 0.025, 0.975)))
    
}

plot(pmeans, seq(chlamydia_dat_m$studnum, 1, -1), 
     xlim=c(-1.5,5), ylim=c(0, 9), xaxt='n', yaxt = 'n', 
     xlab = expression(Slow~clearance~rate~(years^{-1})), ylab = '' )
points(pquantiles[,'50%'], seq(chlamydia_dat_m$studnum, 1, -1) , pch='|')
arrows(x0 = pquantiles[,'25%'], x1 = pquantiles[,'75%'], y0 = seq(chlamydia_dat_m$studnum, 1, -1), lwd=3, length=0)
arrows(x0 = pquantiles[,'2.5%'], x1 = pquantiles[,'97.5%'], y0 = seq(chlamydia_dat_m$studnum, 1, -1), length=0)
text(-1.5, seq(chlamydia_dat_m$studnum, 1, -1), study_names, pos=4)

axis(side=1, at=0:5)

legend('topright', legend=c('Mean', NA, 'Median', NA, '50% CrI', NA, '95% CrI'),
       lty=c(0, NA, 0,NA,1,NA,1), lwd=c(1, NA, 1,NA,3,NA,1), pch=c(1,NA,3,NA,NA,NA,NA), 
       inset = 0.05, cex=0.8)